In [2]:
import os
import time
import pandas as pd
import numpy as np
import networkx as nx
import collections
from scipy import sparse as sp
from scipy.stats import rankdata

import itertools
from itertools import combinations, combinations_with_replacement, cycle
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from util import *

import colorcet as cc

import bokeh
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.models import (Rect, MultiLine, Circle, Span, Label,
                          GraphRenderer, StaticLayoutProvider,
                          NodesAndLinkedEdges,
                          HoverTool, TapTool, ColumnDataSource,
                          LinearColorMapper, LogColorMapper, CategoricalColorMapper,
                          CategoricalMarkerMapper,
                          BoxSelectTool,
                          ColorBar, BasicTicker, BoxZoomTool, FactorRange,
                          Range1d)
from bokeh.models import CategoricalTicker, FixedTicker, BoxAnnotation
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead, LabelSet

from bokeh.transform import transform, factor_cmap, linear_cmap, log_cmap
from bokeh.layouts import row, column, gridplot
output_notebook()

Loading BokehJS ...

In [3]:
def joint_marginal(df, c1, c2, include_fraction=False):
    """Given a dataframe and two columns, return a dataframe with the joint and marginal counts."""
    j = df.value_counts([c1, c2])
    j.name = "joint_count"
    j = j.reset_index()

    m1 = df.value_counts(c1)
    m1.name = f"{c1}_count"
    j = j.merge(m1, left_on=c1, right_index=True)

    m2 = df.value_counts(c2)
    m2.name = f"{c2}_count"
    j = j.merge(m2, left_on=c2, right_index=True)

    if include_fraction:
        j["joint_fraction"] = j["joint_count"] / j["joint_count"].sum()
        j[f"{c1}_fraction"] = j["joint_count"] / j[f"{c1}_count"]
        j[f"{c2}_fraction"] = j["joint_count"] / j[f"{c2}_count"]
    return j


In [4]:
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_" + hemibrain_version
hemibrain_nodes = "inputsoutputs_key.txt"
hemibrain_edges = "inputsoutputs.txt"
#hemibrain_nodes = "only_inputs_key.txt"
#hemibrain_edges = "only_inputs.txt"
#hemibrain_nodes = "only_outputs_key.txt"
#hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

2023 12 15 14:10:42  Hemibrain data set being used: v1.2.1
2023 12 15 14:10:42  Set up directory info and useful lists


In [5]:
from neuprint import Client
from neuprint import fetch_roi_hierarchy, fetch_neurons, NeuronCriteria as NC


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")
pd.set_option('display.max_rows', 200)

2023 12 15 14:10:45  neuprint Client set up as `np_Client`, version 0.1.0
2023 12 15 14:10:45  Loading node dataframe
2023 12 15 14:10:45    (without centroids)
2023 12 15 14:10:45  Adding 'type group'
2023 12 15 14:10:45  Done!
2023 12 15 14:10:45  Loading directed edges from csv
2023 12 15 14:10:45  Done!
2023 12 15 14:10:45  Merging in cell info to edge df
2023 12 15 14:10:45  Done!


In [9]:
HB_node_df

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1003215282,1,1,1,1,1,1,1,CL229_R,CL229,100,...,False,Roughly traced,PDM19,301.0,"[23044, 14981, 11600]","{'INP': {'pre': 87, 'post': 351, 'downstream':...",NaN,"['EPA(R)', 'GOR(R)', 'IB', 'ICL(R)', 'INP', 'S...","['GOR(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S...",CL
1005952640,2,1,1,2,2,2,2,IB058_R,IB058,664,...,False,Roughly traced,PVL20,NaN,NaN,"{'INP': {'pre': 464, 'post': 1327, 'downstream...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...",IB
1006928515,3,1,1,1,3,3,3,CL300_R,CL300,86,...,False,Roughly traced,PVL13,236.0,"[12083, 10523, 16816]","{'INP': {'pre': 79, 'post': 126, 'downstream':...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S...",CL
1007260806,4,1,2,1,4,4,4,CL301_R,CL301,119,...,False,Roughly traced,PVL13,236.0,"[13524, 10108, 16480]","{'INP': {'pre': 40, 'post': 128, 'downstream':...",NaN,"['GOR(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['IB', 'ICL(R)', 'INP', 'PLP(R)', 'SCL(R)', 'S...",CL
1007402796,5,1,1,2,5,5,5,PS119_R,PS119,245,...,False,Roughly traced,PDM16,301.0,"[25364, 12010, 12544]","{'SNP(R)': {'pre': 100, 'post': 50, 'downstrea...",NaN,"['CAN(R)', 'GOR(R)', 'IB', 'ICL(L)', 'ICL(R)',...","['AVLP(R)', 'CAN(R)', 'IB', 'ICL(L)', 'INP', '...",PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988269593,4545,3,5,5,95,13,1785,FB4E_L,FB4E,168,...,False,Roughly traced,AVM08,NaN,NaN,"{'SNP(L)': {'post': 25, 'upstream': 25, 'mito'...",CRELALFB4_1,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FB
988291460,4546,4,4,263,1059,1501,1786,None,None,2,...,NaN,0.5assign,NaN,NaN,NaN,"{'SNP(L)': {'pre': 2, 'post': 1, 'downstream':...",NaN,"['SMP(L)', 'SNP(L)']","['SMP(L)', 'SNP(L)']",None
988567837,4547,5,8,7,13,13,13,FB4G_R,FB4G,785,...,False,Roughly traced,AVM08,NaN,NaN,"{'SNP(R)': {'pre': 6, 'post': 73, 'downstream'...",CRELALFB4_3,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB


In [11]:
from neuprint import fetch_simple_connections
nc = NC(type='SLP316')
neuron_df = fetch_simple_connections("oviIN", nc)
neuron_df

,bodyId_pre,bodyId_post,weight,type_pre,type_post,instance_pre,instance_post,conn_roiInfo


In [35]:
HB_node_df[HB_node_df['celltype']=='aMe24']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
574965048,1097,4,15,11,269,347,306,aMe24(PDM24)_L,aMe24,446,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 439, 'post': 873, 'downstre...",NaN,"['ATL(L)', 'ICL(L)', 'INP', 'SCL(L)', 'SMP(L)'...","['ATL(L)', 'INP', 'SCL(L)', 'SMP(L)', 'SNP(L)']",aMe
5813011660,1234,1,13,15,262,337,358,aMe24_R,aMe24,1346,...,False,Roughly traced,PDM24,355.0,"[17120, 17799, 6112]","{'SNP(R)': {'pre': 706, 'post': 1823, 'downstr...",NaN,"['AME(R)', 'ATL(R)', 'AVLP(R)', 'ICL(R)', 'INP...","['AME(R)', 'AVLP(R)', 'ICL(R)', 'INP', 'LH(R)'...",aMe


In [23]:
s=HB_node_df[HB_node_df['0.0']==4]
x= s[s['0.05']==7]
f=x[x['0.1']==6]
f[f['0.5']==330]
#z=h[h['0.75']==573]
#z[z['1.0']==631]

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
390473497,1160,4,7,6,330,647,720,(PDM12)_L,None,109,...,False,Roughly traced,NaN,295.0,"[27992, 21430, 6128]","{'SNP(L)': {'pre': 101, 'post': 272, 'downstre...",NaN,"['FLA(R)', 'PENP', 'SMP(L)', 'SNP(L)']","['FLA(R)', 'PENP', 'SMP(L)', 'SNP(L)']",None
424193645,1260,4,7,6,330,629,701,None,None,115,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 102, 'post': 1160, 'downstr...",NaN,"['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SIP(L)', 'S...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SIP(L)', 'S...",None
452841283,1316,4,7,6,330,668,746,None,None,378,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 378, 'post': 437, 'downstre...",NaN,"['SMP(L)', 'SNP(L)']","['SMP(L)', 'SNP(L)']",None
454896445,1343,4,7,6,330,712,802,SMP079(ADM04)_L,SMP079,455,...,False,Roughly traced,NaN,292.5,"[25769, 31754, 13064]","{'SNP(L)': {'pre': 336, 'post': 2204, 'downstr...",NaN,"['ATL(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'GOR(L)',...","['ATL(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'GOR(L)',...",SMP
481191030,1385,4,7,6,330,647,720,(PDM12)_L,None,93,...,False,Roughly traced,NaN,291.5,"[27872, 21218, 5280]","{'SNP(L)': {'pre': 87, 'post': 240, 'downstrea...",NaN,"['ATL(L)', 'FLA(R)', 'INP', 'PENP', 'SMP(L)', ...","['FLA(R)', 'PENP', 'SMP(L)', 'SNP(L)']",None
485934965,1435,4,7,6,330,739,845,oviIN_L,oviIN,6542,...,False,Roughly traced,NaN,NaN,NaN,"{'SNP(L)': {'pre': 4184, 'post': 10529, 'downs...",NaN,"['ATL(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'...","['ATL(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'...",oviIN
487286529,1457,4,7,6,330,739,845,(MBDLaxon1),None,906,...,False,Roughly traced,NaN,NaN,NaN,"{'SNP(L)': {'pre': 879, 'post': 546, 'downstre...",NaN,"['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SMP(L)', 'S...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SIP(L)', 'S...",None
517280671,1545,4,7,6,330,573,631,SMP092(ADM04)_L,SMP092,248,...,False,Roughly traced,NaN,292.5,"[27468, 31959, 9536]","{'SNP(L)': {'pre': 78, 'post': 1819, 'downstre...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",SMP
544265926,1628,4,7,6,330,573,631,SMP271(PDL20)_L,SMP271,484,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 477, 'post': 1073, 'downstr...",NaN,"['INP', 'SCL(L)', 'SMP(L)', 'SNP(L)']","['INP', 'SCL(L)', 'SMP(L)', 'SNP(L)']",SMP


In [236]:
chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

g = gridplot(fs, ncols=2)
show(g)

,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,4,7,730,861,766,0.160475,0.847851,0.953003
24,1,7,5,1178,766,0.001099,0.004244,0.006527
25,2,7,5,782,766,0.001099,0.006394,0.006527
75,6,7,1,611,766,0.000220,0.001637,0.001305
13,5,7,23,738,766,0.005056,0.031165,0.030026
32,3,7,2,379,766,0.000440,0.005277,0.002611
9,4,4,82,861,459,0.018026,0.095238,0.178649
7,1,4,181,1178,459,0.039789,0.153650,0.394336
10,2,4,69,782,459,0.015168,0.088235,0.150327
12,6,4,53,611,459,0.011651,0.086743,0.115468


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.05: 147


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,4,6,679,861,721,0.149264,0.788618,0.941748
21,1,6,13,1178,721,0.002858,0.011036,0.018031
15,2,6,26,782,721,0.005716,0.033248,0.036061
103,6,6,1,611,721,0.000220,0.001637,0.001387
59,5,6,2,738,721,0.000440,0.002710,0.002774
...,...,...,...,...,...,...,...,...
186,5,21,1,738,1,0.000220,0.001355,1.000000
73,3,80,2,379,2,0.000440,0.005277,1.000000
115,3,24,1,379,1,0.000220,0.002639,1.000000
116,3,36,1,379,1,0.000220,0.002639,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.1: 263


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,3,9,145,379,145,0.031875,0.382586,1.000000
4,3,13,66,379,69,0.014509,0.174142,0.956522
328,5,13,3,738,69,0.000659,0.004065,0.043478
16,3,18,38,379,56,0.008353,0.100264,0.678571
263,4,18,4,861,56,0.000879,0.004646,0.071429
...,...,...,...,...,...,...,...,...
871,6,919,1,611,1,0.000220,0.001637,1.000000
876,6,849,1,611,1,0.000220,0.001637,1.000000
882,6,916,1,611,1,0.000220,0.001637,1.000000
904,6,925,1,611,1,0.000220,0.001637,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.5: 1059


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,12,80,379,95,0.017586,0.211082,0.842105
1670,4,12,1,861,95,0.000220,0.001161,0.010526
223,5,12,4,738,95,0.000879,0.005420,0.042105
41,6,12,10,611,95,0.002198,0.016367,0.105263
1,3,9,61,379,61,0.013410,0.160950,1.000000
...,...,...,...,...,...,...,...,...
1255,6,431,1,611,1,0.000220,0.001637,1.000000
1274,6,196,1,611,1,0.000220,0.001637,1.000000
1275,6,173,1,611,1,0.000220,0.001637,1.000000
1278,6,150,1,611,1,0.000220,0.001637,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.75: 1501


In [199]:
r= HB_node_df[HB_node_df['0.0']==3]
f = r[r['0.05']==5]
z= f[f['0.1']==5]
z[z['0.5']==9]
#h[h['0.75']==496]


,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1008024276,9,3,5,5,9,9,9,FB5N_R,FB5N,499,...,False,Roughly traced,AVM08,472.5,"[19178, 29711, 37312]","{'SNP(L)': {'post': 5, 'upstream': 5, 'mito': ...",SMPCREFB5_4,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1008378609,14,3,5,5,9,14,9,FB5Y_R,FB5Y,525,...,False,Roughly traced,PDL05,286.0,"[8843, 14301, 12328]","{'SNP(R)': {'pre': 1, 'post': 1066, 'downstrea...",SMPSIPFB5d_2,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'EB', 'E...","['CX', 'FB', 'FB-column3', 'FBl5', 'FBl6', 'SM...",FB
1008391068,15,3,5,5,9,9,14,FC2B(FQ7)_C6_L,FC2B,219,...,False,Roughly traced,PDM03,325.5,"[29681, 16321, 7328]","{'CX': {'pre': 167, 'post': 2135, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1008693462,18,3,5,5,9,12,17,FC2C(FQ7)_C4_R,FC2C,146,...,False,Roughly traced,PDM03,301.0,"[20309, 14072, 10768]","{'CX': {'pre': 107, 'post': 1234, 'downstream'...",FB1d367CRE,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FC
1008693605,19,3,5,5,9,9,18,FC2C(FQ7)_C7_L,FC2C,126,...,False,Roughly traced,PDM03,291.0,"[32830, 14210, 12336]","{'CX': {'pre': 91, 'post': 1051, 'downstream':...",FB1d367CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1009371535,23,3,5,5,9,9,21,FC2B(FQ7)_C3_L,FC2B,169,...,False,Roughly traced,PDM01,277.5,"[28298, 18341, 7600]","{'CX': {'pre': 130, 'post': 2051, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1010437434,29,3,5,5,9,23,18,FC2C(FQ7)_C7_L,FC2C,118,...,False,Roughly traced,PDM03,291.0,"[33870, 12655, 13640]","{'CX': {'pre': 83, 'post': 980, 'downstream': ...",FB1d367CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1011068028,31,3,5,5,9,25,28,FB6R_R,FB6R,185,...,False,Roughly traced,PDM25,301.0,"[20012, 16808, 8384]","{'SNP(R)': {'pre': 1, 'post': 411, 'downstream...",SMPSIPFB6_7,"['CX', 'FB', 'FB-column3', 'FBl5', 'FBl6', 'FB...","['CX', 'FB', 'FB-column3', 'FBl5', 'FBl6', 'FB...",FB
1014102651,47,3,5,5,9,9,43,FS1A(FQ9)_C2_L,FS1A,89,...,False,Roughly traced,PDM01,301.0,"[26807, 16994, 11456]","{'SNP(R)': {'pre': 45, 'post': 34, 'downstream...",FB2-6SMPSMP,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FS


In [204]:
r= HB_node_df[HB_node_df['0.0']==3]
f = r[r['0.05']==5]
z= f[f['0.1']==5]
h = z[z['0.5']==9]
x=h[h['0.75']==9]
x[x['1.0']==14]

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1008391068,15,3,5,5,9,9,14,FC2B(FQ7)_C6_L,FC2B,219,...,False,Roughly traced,PDM03,325.5,"[29681, 16321, 7328]","{'CX': {'pre': 167, 'post': 2135, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1014474192,59,3,5,5,9,9,14,FS1A(FQ9)_C2_L,FS1A,112,...,False,Roughly traced,PDM01,301.0,"[27300, 17574, 10448]","{'SNP(L)': {'pre': 31, 'post': 18, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1041071176,118,3,5,5,9,9,14,FC2B(FQ7)_C2_L,FC2B,185,...,False,Roughly traced,PDM01,325.5,"[25834, 20180, 7336]","{'CX': {'pre': 141, 'post': 2093, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1070102614,185,3,5,5,9,9,14,FB5G_L,FB5G,307,...,False,Roughly traced,AVM10,NaN,NaN,"{'SNP(L)': {'pre': 36, 'post': 271, 'downstrea...",SMPSIPFB56,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FB
1071760480,196,3,5,5,9,9,14,FC2C(FQ7)_C2_L,FC2C,173,...,False,Roughly traced,PDM01,325.5,"[26931, 21788, 7896]","{'SNP(R)': {'pre': 16, 'post': 31, 'downstream...",FB1d367CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1072533704,202,3,5,5,9,9,14,FC2C(FQ7)_C8_R,FC2C,144,...,False,Roughly traced,PDM01,301.0,"[25619, 13451, 12464]","{'SNP(L)': {'pre': 8, 'post': 9, 'downstream':...",FB1d367CRE,"['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FBl1',...",FC
1134849078,295,3,5,5,9,9,14,FC2B(FQ7)_C1_L,FC2B,170,...,False,Roughly traced,PDM01,316.0,"[27433, 20299, 9392]","{'SNP(R)': {'pre': 9, 'post': 15, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1164545788,345,3,5,5,9,9,14,FC2B(FQ7)_C2_L,FC2B,246,...,False,Roughly traced,PDM01,325.5,"[26400, 18401, 7648]","{'CX': {'pre': 184, 'post': 2808, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1196573432,376,3,5,5,9,9,14,FC2B(FQ7)_C2_R,FC2B,215,...,False,Roughly traced,PDM04,306.0,"[19243, 9331, 16440]","{'SNP(L)': {'pre': 1, 'post': 5, 'downstream':...",FB1d356CRE,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FC


In [7]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html")
save(g, title='Joint Marginal Analysis (full)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html'

In [31]:
# Inputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_inputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_inputs_" + hemibrain_version
hemibrain_nodes = "only_inputs_key.txt"
hemibrain_edges = "only_inputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

k = gridplot(fs, ncols=2)

2023 08 03 16:52:34  Hemibrain data set being used: v1.2.1
2023 08 03 16:52:34  Set up directory info and useful lists
2023 08 03 16:52:34  neuprint Client set up as `np_Client`, version 0.1.0
2023 08 03 16:52:34  Loading node dataframe
2023 08 03 16:52:34    (without centroids)
2023 08 03 16:52:34  Adding 'type group'
2023 08 03 16:52:34  Done!
2023 08 03 16:52:34  Loading directed edges from csv
2023 08 03 16:52:34  Done!
2023 08 03 16:52:34  Merging in cell info to edge df
2023 08 03 16:52:34  Done!


,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,1,13,374,519,392,0.148945,0.720617,0.954082
16,5,13,13,387,392,0.005177,0.033592,0.033163
31,4,13,3,302,392,0.001195,0.009934,0.007653
41,2,13,2,716,392,0.000796,0.002793,0.005102
9,1,1,57,519,271,0.022700,0.109827,0.210332
28,4,1,4,302,271,0.001593,0.013245,0.014760
4,2,1,210,716,271,0.083632,0.293296,0.774908
10,1,2,28,519,326,0.011151,0.053950,0.085890
34,5,2,3,387,326,0.001195,0.007752,0.009202
2,3,2,293,587,326,0.116687,0.499148,0.898773


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.05: 101


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,5,20,283,387,288,0.112704,0.731266,0.982639
31,3,20,5,587,288,0.001991,0.008518,0.017361
14,5,8,27,387,221,0.010753,0.069767,0.122172
33,4,8,4,302,221,0.001593,0.013245,0.018100
101,1,8,1,519,221,0.000398,0.001927,0.004525
23,2,8,12,716,221,0.004779,0.016760,0.054299
6,3,8,177,587,221,0.070490,0.301533,0.800905
16,5,6,24,387,281,0.009558,0.062016,0.085409
47,4,6,2,302,281,0.000796,0.006623,0.007117
3,1,6,195,519,281,0.077658,0.375723,0.693950


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.1: 89


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,3,5,150,587,151,0.059737,0.255537,0.993377
507,2,5,1,716,151,0.000398,0.001397,0.006623
2,3,10,46,587,46,0.018319,0.078365,1.000000
18,3,19,19,587,19,0.007567,0.032368,1.000000
19,3,263,18,587,18,0.007168,0.030664,1.000000
...,...,...,...,...,...,...,...,...
465,4,458,1,302,1,0.000398,0.003311,1.000000
467,4,462,1,302,1,0.000398,0.003311,1.000000
477,4,466,1,302,1,0.000398,0.003311,1.000000
479,4,480,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.5: 533


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,129,587,131,0.051374,0.219761,0.984733
493,2,5,2,716,131,0.000796,0.002793,0.015267
3,3,10,24,587,24,0.009558,0.040886,1.000000
8,3,309,14,587,14,0.005575,0.023850,1.000000
17,3,19,10,587,10,0.003982,0.017036,1.000000
...,...,...,...,...,...,...,...,...
720,4,717,1,302,1,0.000398,0.003311,1.000000
721,4,716,1,302,1,0.000398,0.003311,1.000000
748,4,225,1,302,1,0.000398,0.003311,1.000000
920,4,279,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.75: 780


In [112]:
show(k)

In [170]:
r=HB_node_df[HB_node_df['0.0']==3]
f= r[r['0.05']==2]
z=f[f['0.1']==2]
h=z[z['0.5']==5]
x=h[h['0.75']==5]
x[x['1.0']==5]
# Either 5, 8 9

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1008024276,5,3,2,2,5,5,5,FB5N_R,FB5N,499,...,False,Roughly traced,AVM08,472.5,"[19178, 29711, 37312]","{'SNP(L)': {'post': 5, 'upstream': 5, 'mito': ...",SMPCREFB5_4,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1008391068,9,3,2,2,5,5,5,FC2B(FQ7)_C6_L,FC2B,219,...,False,Roughly traced,PDM03,325.5,"[29681, 16321, 7328]","{'CX': {'pre': 167, 'post': 2135, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1039068185,59,3,2,2,5,5,5,FB5O_L,FB5O,574,...,False,Roughly traced,AVM10,NaN,NaN,"{'SNP(L)': {'pre': 25, 'post': 270, 'downstrea...",SMPCREFB5_5,"['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...","['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...",FB
1039400642,61,3,2,2,5,5,5,FB5O_R,FB5O,579,...,False,Roughly traced,AVM10,292.5,"[12926, 31418, 17584]","{'SNP(R)': {'pre': 24, 'post': 515, 'downstrea...",SMPCREFB5_5,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1070422316,103,3,2,2,5,5,5,FC2B(FQ7)_C3_L,FC2B,237,...,False,Roughly traced,PDM01,325.5,"[28778, 18075, 8016]","{'CX': {'pre': 186, 'post': 2696, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1071121755,105,3,2,2,5,5,5,FB6W_R,FB6W,503,...,False,Roughly traced,AVM10,336.5,"[12515, 33697, 19328]","{'SNP(R)': {'pre': 14, 'post': 266, 'downstrea...",CRESMPFB6_2,"['BU(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX',...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1164545788,191,3,2,2,5,5,5,FC2B(FQ7)_C2_L,FC2B,246,...,False,Roughly traced,PDM01,325.5,"[26400, 18401, 7648]","{'CX': {'pre': 184, 'post': 2808, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1257629175,234,3,2,2,5,5,5,FC2B(FQ7)_C1_L,FC2B,217,...,False,Roughly traced,PDM01,325.5,"[26437, 21157, 7688]","{'SNP(R)': {'pre': 3, 'post': 11, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
486842208,898,3,2,2,5,5,5,FB5C_R,FB5C,533,...,False,Roughly traced,AVM10,311.5,"[12096, 31738, 19024]","{'SNP(R)': {'pre': 78, 'post': 528, 'downstrea...",SMPCREFB5_1,"['BU(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX',...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB


In [9]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_inputs.html")
save(k, title='Joint Marginal Analysis (Inputs)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_inputs.html'

In [7]:
# outputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_outputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_outputs_" + hemibrain_version
hemibrain_nodes = "only_outputs_key.txt"
hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

i = gridplot(fs, ncols=2)

2023 07 24 14:53:11  Hemibrain data set being used: v1.2.1
2023 07 24 14:53:11  Set up directory info and useful lists
2023 07 24 14:53:14  neuprint Client set up as `np_Client`, version 0.1.0
2023 07 24 14:53:14  Loading node dataframe
2023 07 24 14:53:14    (without centroids)
2023 07 24 14:53:14  Adding 'type group'
2023 07 24 14:53:14  Done!
2023 07 24 14:53:14  Loading directed edges from csv
2023 07 24 14:53:14  Done!
2023 07 24 14:53:14  Merging in cell info to edge df
2023 07 24 14:53:15  Done!


,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,4,5,597,778,602,0.173597,0.767352,0.991694
37,5,5,3,566,602,0.000872,0.005300,0.004983
52,6,5,2,550,602,0.000582,0.003636,0.003322
8,4,3,50,778,222,0.014539,0.064267,0.225225
9,1,3,46,712,222,0.013376,0.064607,0.207207
...,...,...,...,...,...,...,...,...
125,6,23,1,550,1,0.000291,0.001818,1.000000
126,6,26,1,550,1,0.000291,0.001818,1.000000
127,6,27,1,550,1,0.000291,0.001818,1.000000
208,6,159,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.05: 160


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,4,5,621,778,632,0.180576,0.798201,0.982595
36,2,5,4,601,632,0.001163,0.006656,0.006329
54,5,5,2,566,632,0.000582,0.003534,0.003165
41,6,5,4,550,632,0.001163,0.007273,0.006329
255,3,5,1,232,632,0.000291,0.004310,0.001582
...,...,...,...,...,...,...,...,...
138,6,177,1,550,1,0.000291,0.001818,1.000000
141,6,18,1,550,1,0.000291,0.001818,1.000000
143,6,13,1,550,1,0.000291,0.001818,1.000000
263,6,201,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.1: 205


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,5,15,136,566,157,0.039546,0.240283,0.866242
38,4,15,13,778,157,0.003780,0.016710,0.082803
694,6,15,1,550,157,0.000291,0.001818,0.006369
170,1,15,4,712,157,0.001163,0.005618,0.025478
284,2,15,3,601,157,0.000872,0.004992,0.019108
...,...,...,...,...,...,...,...,...
874,2,508,1,601,1,0.000291,0.001664,1.000000
970,2,163,1,601,1,0.000291,0.001664,1.000000
971,2,181,1,601,1,0.000291,0.001664,1.000000
975,2,191,1,601,1,0.000291,0.001664,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.5: 745


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,53,232,53,0.015411,0.228448,1.000000
3,3,93,21,232,21,0.006106,0.090517,1.000000
6,3,95,17,232,17,0.004943,0.073276,1.000000
9,3,18,16,232,16,0.004653,0.068966,1.000000
15,3,52,13,232,14,0.003780,0.056034,0.928571
...,...,...,...,...,...,...,...,...
1038,6,910,1,550,1,0.000291,0.001818,1.000000
1040,6,909,1,550,1,0.000291,0.001818,1.000000
1044,6,886,1,550,1,0.000291,0.001818,1.000000
1047,6,879,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.75: 1098


In [11]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html")
save(i, title='Join Marginal Analysis (Outputs)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html'

In [8]:
HB_node_df[HB_node_df['celltype']=='SMP550']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
452689494,789,6,32,5,234,437,498,SMP550_R,SMP550,1414,...,False,Roughly traced,PDM29,347.0,"[19511, 18367, 6016]","{'SNP(R)': {'pre': 1248, 'post': 3275, 'downst...",pSP7 candidates,"['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...","['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...",SMP
579700707,1306,4,5,5,335,478,551,SMP550(PDM29)_L,SMP550,992,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 981, 'post': 1089, 'downstr...",pSP7 candidates,"['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']","['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']",SMP


In [275]:
r=HB_node_df[HB_node_df['0.0']==6]
f= r[r['0.05']==9]
z=f[f['0.1']==16]
z[z['0.5']==22]
#h[h['0.75']==129]


,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1015479949,43,6,9,16,22,35,35,CRE003_a_R,CRE003_a,309,...,False,Roughly traced,ADL06,306.5,"[12340, 32294, 20200]","{'SNP(R)': {'pre': 20, 'post': 341, 'downstrea...",aSP14b candidates,"['ATL(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'...","['ATL(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'...",CRE
1723506321,343,6,9,16,22,218,246,M_l2PNm14_R,M_l2PNm14,957,...,False,Roughly traced,AVM03,441.5,"[13727, 33545, 32552]","{'SNP(R)': {'pre': 465, 'post': 353, 'downstre...",NaN,"['AL(R)', 'AL-D(R)', 'AL-DA2(R)', 'AL-DC1(R)',...","['AL(R)', 'AL-DC1(R)', 'AL-DC2(R)', 'AL-DC4(R)...",M_l
355086967,562,6,9,16,22,160,384,SMP211_R,SMP211,177,...,False,Roughly traced,PDL06,285.0,"[6295, 18095, 5392]","{'SNP(R)': {'pre': 59, 'post': 223, 'downstrea...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'SCL(R)...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'SIP(R)...",SMP
388280322,637,6,9,16,22,372,427,SMP448_R,SMP448,172,...,False,Roughly traced,PDM11,240.0,"[19409, 22871, 3792]","{'SNP(R)': {'pre': 138, 'post': 688, 'downstre...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'RUB(R)...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'SCL(R)...",SMP
417834021,688,6,9,16,22,394,384,LHPD2a4_a_R,LHPD2a4_a,318,...,False,Roughly traced,PDL15,299.5,"[6148, 18697, 6352]","{'SNP(R)': {'pre': 211, 'post': 528, 'downstre...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...",LHPD
449205588,762,6,9,16,22,426,489,LHPD2a4_b_R,LHPD2a4_b,302,...,False,Roughly traced,PDL15,341.0,"[4210, 22128, 6624]","{'SNP(R)': {'pre': 180, 'post': 354, 'downstre...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...",LHPD
449205611,763,6,9,16,22,372,427,LHPD2a4_a_R,LHPD2a4_a,319,...,False,Roughly traced,PDL15,288.0,"[4298, 21397, 6368]","{'SNP(R)': {'pre': 222, 'post': 406, 'downstre...",NaN,"['AVLP(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'IN...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...",LHPD
449209971,764,6,9,16,22,372,427,SIP049_R,SIP049,269,...,False,Roughly traced,PDL15,318.5,"[4010, 22141, 7904]","{'SNP(R)': {'pre': 242, 'post': 335, 'downstre...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'LH(R)'...",SIP
484130600,873,6,9,16,22,467,535,SMP145_R,SMP145,2264,...,False,Roughly traced,ADM10,269.5,"[18922, 31643, 7680]","{'SNP(R)': {'pre': 760, 'post': 2086, 'downstr...",NaN,"['ATL(L)', 'ATL(R)', 'AVLP(R)', 'CA(R)', 'CRE(...","['AL(R)', 'ATL(L)', 'ATL(R)', 'BU(R)', 'CA(R)'...",SMP


In [278]:
HB_node_df[HB_node_df['celltype']=='pC1e']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
514850616,974,1,1,1,312,494,578,pC1e_R,pC1e,697,...,False,Traced,PDM09,356.0,"[23338, 10397, 17648]","{'SNP(R)': {'pre': 387, 'post': 1777, 'downstr...",NaN,"['AOTU(R)', 'AVLP(R)', 'EPA(R)', 'ICL(R)', 'IN...","['AOTU(R)', 'AVLP(R)', 'EPA(R)', 'ICL(R)', 'IN...",pC


In [229]:
h = z[z['0.5']==4]
x=h[h['0.75']==5]
x[x['1.0']==61]

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1039068185,68,3,4,4,4,5,61,FB5O_L,FB5O,574,...,False,Roughly traced,AVM10,NaN,NaN,"{'SNP(L)': {'pre': 25, 'post': 270, 'downstrea...",SMPCREFB5_5,"['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...","['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...",FB
1039400642,73,3,4,4,4,5,61,FB5O_R,FB5O,579,...,False,Roughly traced,AVM10,292.5,"[12926, 31418, 17584]","{'SNP(R)': {'pre': 24, 'post': 515, 'downstrea...",SMPCREFB5_5,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1070422316,119,3,4,4,4,5,61,FC2B(FQ7)_C3_L,FC2B,237,...,False,Roughly traced,PDM01,325.5,"[28778, 18075, 8016]","{'CX': {'pre': 186, 'post': 2696, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
1071121755,121,3,4,4,4,5,61,FB6W_R,FB6W,503,...,False,Roughly traced,AVM10,336.5,"[12515, 33697, 19328]","{'SNP(R)': {'pre': 14, 'post': 266, 'downstrea...",CRESMPFB6_2,"['BU(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX',...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
1164545788,216,3,4,4,4,5,61,FC2B(FQ7)_C2_L,FC2B,246,...,False,Roughly traced,PDM01,325.5,"[26400, 18401, 7648]","{'CX': {'pre': 184, 'post': 2808, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC
486842208,910,3,4,4,4,5,61,FB5C_R,FB5C,533,...,False,Roughly traced,AVM10,311.5,"[12096, 31738, 19024]","{'SNP(R)': {'pre': 78, 'post': 528, 'downstrea...",SMPCREFB5_1,"['BU(L)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX',...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FB
548571023,1150,3,4,4,4,5,61,FB5B_R,FB5B,169,...,False,Roughly traced,PDM25,301.0,"[20977, 15883, 9488]","{'SNP(R)': {'pre': 1, 'post': 387, 'downstream...",SMPSIPFB5d_1,"['CX', 'FB', 'FB-column3', 'FBl5', 'FBl6', 'FB...","['CX', 'FB', 'FB-column3', 'FBl5', 'FBl6', 'SM...",FB
5812983636,1372,3,4,4,4,5,61,FB5C_L,FB5C,587,...,False,Roughly traced,AVM10,NaN,NaN,"{'SNP(L)': {'pre': 65, 'post': 364, 'downstrea...",SMPCREFB5_1,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['BU(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...",FB
5813057464,1610,3,4,4,4,5,61,FC2B(FQ7)_C7_L,FC2B,189,...,False,Roughly traced,PDM03,291.0,"[34198, 11737, 13872]","{'CX': {'pre': 144, 'post': 1884, 'downstream'...",FB1d356CRE,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...",FC


In [223]:
HB_node_df[HB_node_df['celltype']=='SMP550']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
452689494,789,6,32,5,234,437,498,SMP550_R,SMP550,1414,...,False,Roughly traced,PDM29,347.0,"[19511, 18367, 6016]","{'SNP(R)': {'pre': 1248, 'post': 3275, 'downst...",pSP7 candidates,"['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...","['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...",SMP
579700707,1306,4,5,5,335,478,551,SMP550(PDM29)_L,SMP550,992,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 981, 'post': 1089, 'downstr...",pSP7 candidates,"['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']","['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']",SMP


In [6]:
# Pulled from Prof G's code on github (https://github.com/Gutierrez-lab/oviIN-analyses-gabrielle/blob/main/modular_sandbox.ipynb)
def modularity_merge(df1,df2,suf1,suf2):
    """Given two modularity dataframes, merge them along shared body IDs. Pass in suffixes for the columns as strings."""
    merged_mod_df = df1.merge(df2, left_on='id', right_on='id', suffixes=[suf1, suf2])
    #merged_mod_df = df1.merge(df2, left_on='id', right_on='id', suffixes=['_oviHB', '_wholeHB'])
    return merged_mod_df

# Defining path to whole brain data
preproc_dir = "hemibrain/preprocessed-" + "v1.2"
preproc_nodes = "preprocessed_nodes.csv"

# Creating dataframe
WB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
WB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)

# Print dataframe
WB_node_df

,0.75,0.05,0.1,0.25,0.5,1.0,0.0,instance,celltype,pre,...,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,inputRois,outputRois,roiInfo,type_group
id,,,,,,,,,,,,,,,,,,,,,
200326126,1,1,1,1,1,1,1,CL141_L,CL141,395,...,Traced,False,Roughly traced,PDL20,251.0,"[3844, 14565, 15568]","['AVLP(R)', 'IB', 'ICL(R)', 'INP', 'LH(R)', 'M...","['AVLP(R)', 'ICL(R)', 'INP', 'LH(R)', 'MB(+ACA...","{'SNP(R)': {'pre': 17, 'post': 19, 'downstream...",NaN
202916528,2,2,2,2,2,2,2,FS4C(FQ12)_C4_R,FS4C,95,...,Traced,False,Roughly traced,PDM03,301.0,"[20091, 12266, 12208]","['CX', 'FB', 'FB-column3', 'FBl1', 'FBl2', 'FB...","['CX', 'FB', 'FBl7', 'SMP(L)', 'SNP(L)']","{'SNP(L)': {'pre': 93, 'post': 35, 'downstream...",NaN
203253072,3,2,3,3,3,3,2,FS4A(AB)(FQ12)_C3_R,FS4A,62,...,Traced,False,Roughly traced,PDM03,236.0,"[17809, 10233, 14208]","['AB(R)', 'CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB'...","['AB(R)', 'CX', 'FB', 'FB-column3', 'FBl1', 'F...","{'SNP(L)': {'pre': 52, 'post': 30, 'downstream...",NaN
203253253,4,3,3,4,4,4,2,SMP505(PDM22)_L,SMP505,415,...,Traced,False,Roughly traced,NaN,351.0,"[32168, 13757, 13336]","['ATL(L)', 'CRE(-RUB)(L)', 'CRE(L)', 'ICL(L)',...","['SIP(L)', 'SIP(R)', 'SLP(R)', 'SMP(L)', 'SMP(...","{'SNP(L)': {'pre': 150, 'post': 2282, 'downstr...",SMP
203257652,5,2,3,2,2,2,2,FS4C(FQ12)_C6_R,FS4C,65,...,Traced,False,Roughly traced,PDM02,301.0,"[22758, 16749, 10240]","['CX', 'FB', 'FBl1', 'FBl2', 'FBl3', 'FBl4', '...","['SIP(L)', 'SMP(L)', 'SNP(L)']","{'SNP(L)': {'pre': 65, 'post': 36, 'downstream...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7112579856,2322,1,1,157,1064,3243,1,LC20,LC20,319,...,Traced,False,Roughly traced,NaN,301.5,"[7883, 23524, 33072]","['LO(R)', 'OL(R)', 'PLP(R)', 'VLNP(R)']","['LO(R)', 'OL(R)', 'PLP(R)', 'VLNP(R)']","{'OL(R)': {'pre': 268, 'post': 594, 'downstrea...",NaN
7112615127,2670,9,19,46,226,3110,6,AVLP335_R,AVLP335,362,...,Traced,False,Roughly traced,AVL13,NaN,NaN,"['AVLP(R)', 'PVLP(R)', 'VLNP(R)']","['AVLP(R)', 'PVLP(R)', 'VLNP(R)']","{'VLNP(R)': {'pre': 362, 'post': 577, 'downstr...",NaN
7112617294,2386,9,19,46,226,3362,6,AVLP331_R,AVLP331,285,...,Traced,False,Roughly traced,AVL13,NaN,NaN,"['AVLP(R)', 'PVLP(R)', 'VLNP(R)']","['AVLP(R)', 'PVLP(R)', 'VLNP(R)']","{'VLNP(R)': {'pre': 285, 'post': 661, 'downstr...",NaN


In [7]:
# Defining base res and suffixes
res = '0.0'
df1_suf = '_oviHB'
df2_suf = '_wholeHB'

# Using merge function from code cell above
mod_merge_df = modularity_merge(HB_node_df[[res]],WB_node_df[[res]],df1_suf,df2_suf)
mod_merge_df

,0.0_aplHB,0.0_wholeHB
id,,
1003215282,1,6
1005952640,1,1
1006928515,1,1
1007260806,1,1
1007402796,1,1
...,...,...
987877232,3,2
988269593,3,2
988567837,5,2


In [8]:
chi1 = res + df1_suf
chi2 = res + df2_suf

jm = joint_marginal(mod_merge_df, chi1, chi2, include_fraction=True)

# sort the clusters on the y axis to get a more "diagonal" plot
yrange = jm.sort_values([f"{chi2}_fraction"], ascending=False).groupby(chi2).agg({chi1: "first", f"{chi2}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index

# make a bokeh figure
f = figure(title=f"Clusters at chi2 = {chi2} vs. clusters at chi1 = {chi1}",
x_range=FactorRange(factors=[str(i + 1) for i in range(jm[chi1].max())]),
y_range=FactorRange(factors=[str(y) for y in yrange]),
width=600, height=700)
jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
jm["y"] = jm[chi2].apply(str)

f.rect(x="x", y="y", width=f"{chi1}_fraction", height=f"{chi2}_fraction", source=jm)
f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                f"Fraction of {chi2}": f"@{{{chi2}_fraction}}{{2.%%}}",
                                f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}"}))
f.xaxis.axis_label = 'Cluster in ' +chi1
f.yaxis.axis_label = 'Cluster in ' +chi2

show(f)